In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from os import listdir
from os.path import isfile, join
from os.path import getmtime
from directories import *
import time
#import po_ordini

In [2]:
data_agg = dt.datetime.fromtimestamp(getmtime(bpm_export_file))

bpm = pd.read_csv(bpm_export_file, sep=";", parse_dates=["Data Caricamento (in BPM)"], date_parser=lambda x: dt.datetime.strptime(x, '%d/%m/%Y'), decimal=",", on_bad_lines="skip")


C:\Users\Raffaele.Sportiello\AppData\Local\Temp\1\ipykernel_25696\3276050917.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  bpm = pd.read_csv(bpm_export_file, sep=";", parse_dates=["Data Caricamento (in BPM)"], date_parser=lambda x: dt.datetime.strptime(x, '%d/%m/%Y'), decimal=",", on_bad_lines="skip")


In [4]:
par_file_path["agente_ordine"]

'C:\\Users\\Raffaele.Sportiello\\OneDrive - Wolters Kluwer\\Documents\\Dashboard inflow\\Parametriche\\Agente_ordine.xlsx'

In [6]:
par = {}
for file in parametriche_files:
    file_name=file.split(".")[0]
    par[f"{file_name}"] = pd.read_excel(parametriche+"\\"+file)


agente = pd.DataFrame(par["Agente_ordine"][["Codice agente", "Agente dell'ORDINE", "RSM Agente", "Sales Director"]])

In [9]:
agente = pd.DataFrame(par["Agente_ordine"][["Codice agente", "Agente dell'ORDINE", "RSM Agente", "Sales Director"]])

,Codice agente,Agente dell'ORDINE
0,5,IPSODATA GENOVA SNC
1,8,CANDIDO CARLA &C SAS
2,11,DUEGI S.A.S.
3,12,GIAMBANET S.A.S.
4,15,EVOLUZIONE S.A.S.
...,...,...
1169,X060,BLUWEB S.R.L.
1170,K105,DCA SOFTWARE S.R.L.
1171,K179,SOFTMAINT S.R.L.
1172,K224,D' AMICO FABIO


In [158]:
def df_style(val):
    return "font-weight: bold"

last_row = bpm_gb_sintesi.loc[
    bpm_gb_sintesi.index == ("Totale",""),:]

bpm_gb_sintesi.style.format(thousands=".", precision=0, subset=["Valore Totale Ordine","Importo Licenza","Importo Servizi","Importo HW","Totale_Importi"])
#applymap(df_style, subset=last_row)

In [1]:
import pandas as pd
import os
import numpy as np
import datetime as dt
from os import listdir
from os.path import isfile, join
from os.path import getmtime
from directories import bpm_folder, bpm_export_file, par_file_path
import time 

In [5]:
data_agg = dt.datetime.fromtimestamp(getmtime(bpm_export_file))

In [6]:
bpm = pd.read_csv(bpm_export_file, sep=";", parse_dates=["Data Caricamento (in BPM)"], date_parser=lambda x: dt.datetime.strptime(x, '%d/%m/%Y'), decimal=",", on_bad_lines="skip")

agente = pd.read_excel(par_file_path["agente_ordine"], usecols=["Codice agente", "Agente dell'ORDINE", "RSM Agente", "Sales Director"], dtype={"Codice agente":str})

sales_director = pd.read_excel(par_file_path["sales_director"], usecols=["Sales Director", "Area territoriale", "Rete"])

columns = ['Data Caricamento (in BPM)',  'Stato Lavorazione','Cod. Pratica (BPM)', 'Nr. Ordine','Inserito da', 'Sales Office', 'Codice Cliente SAP', 'Partita Iva/CF','Nome Cliente', 'Data Documento', 'Ordine Omaggio','Valore Totale Ordine', 'Importo HW', 'Importo Licenza','Importo Servizi', 'Note', 'Ord. Apparound','Automatico SAP', 'Opportunity Id', "Sales Director", "Rete", "Area territoriale", "Agente dell'ORDINE", "RSM Agente"]

bpm_def = ((bpm.loc[:,:].merge(agente, how="left",left_on="Sales Office", right_on="Codice agente"))\
    .merge(sales_director, left_on="Sales Director_y", right_on="Sales Director", how="left"))\
        .loc[:,columns]

In [7]:
bpm_gb_sintesi = bpm_def.groupby(["Rete", "Area territoriale"], dropna=False).agg({"Valore Totale Ordine":sum, "Importo Licenza":sum, "Importo Servizi":sum, "Importo HW":sum}).assign(Totale_Importi=lambda df_: df_[["Importo Licenza", "Importo Servizi", "Importo HW"]].sum(axis=1))
bpm_gb_sintesi_totale = bpm_gb_sintesi.agg("sum").rename("Totale").to_frame().T
bpm_gb_sintesi_totale.index = pd.MultiIndex.from_tuples([("Totale", '')], names=('Rete', 'Area territoriale'))
bpm_gb_sintesi = pd.concat([bpm_gb_sintesi, bpm_gb_sintesi_totale], axis=0)

In [8]:
bpm_gb_numero_ord = bpm_def.groupby(["Rete", "Area territoriale", "Data Caricamento (in BPM)", "Inserito da", "Nr. Ordine", "Stato Lavorazione"], dropna=False).agg({"Valore Totale Ordine":sum, "Importo Licenza":sum, "Importo Servizi":sum, "Importo HW":sum})

In [9]:
registro_ordini = bpm_folder+r"\Registro ordini inseriti in PPM.xlsx"
with pd.ExcelWriter(registro_ordini, datetime_format='YYYY-MM-DD') as writer:
    workbook  = writer.book
    worksheet = workbook.add_worksheet('Ordini caricati')
    worksheet.insert_image('A1', r'C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Speciali\SFvsSAP.png')

    bpm_gb_sintesi.to_excel(writer, sheet_name="Sintesi", float_format="%.0f")
    
    bpm_gb_numero_ord.to_excel(writer, sheet_name="Numero ord", float_format="%.0f")
    bpm_def.to_excel(writer, sheet_name="Dettaglio", index=False)

In [10]:
import openpyxl

# Apri il file Excel con openpyxl
workbook = openpyxl.load_workbook(bpm_folder+r"\Registro ordini inseriti in PPM.xlsx")
worksheet = workbook['Sintesi']

worksheet["I1"] = "Aggiornamento al:"
worksheet["J1"] = data_agg

# Cerca la riga contenente "Totale" e applica lo stile "bold"
for row in worksheet.iter_rows():
    for cell in row:
        if cell.value == 'Totale':
            for c in row:
                c.font = openpyxl.styles.Font(bold=True)


for col in worksheet.columns:
    for cell in col:
        if cell.row > 1:
            cell.number_format = '#,##0'


# Salva il file Excel modificato
workbook.save(bpm_folder+r"\Registro ordini inseriti in PPM.xlsx")

In [7]:

import win32com.client as client
excel = client.gencache.EnsureDispatch('Excel.Application')
wb = excel.Workbooks.Open(registro_ordini)

for sheet in wb.Sheets:
      
    ws = wb.Worksheets(sheet.Name)
    ws.Columns.AutoFit()

wb.Save()
excel.Application.Quit()


time.sleep(5)


#############################################################################################################
# Email
#

attachments = [registro_ordini,r'C:\Users\Raffaele.Sportiello\OneDrive - Wolters Kluwer\Documents\Dashboard inflow\Dashboard inflow canali e prodotti\Speciali\SFvsSAP.png']

html_body = """
    <div>
          <p>Ciao a tutti,<br><br>
            in allegato il file aggiornato.<br><br>
            Un saluto,<br>Raffaele<br><br></p>
    </div>
"""


outlook = client.Dispatch('Outlook.Application')
message = outlook.CreateItem(0)
message.To = "Cristiano.Mozzolani@wolterskluwer.com;Andrea.Ferrara@wolterskluwer.com;rossella.specia@wolterskluwer.com;Gabriele.Albarello@wolterskluwer.com;Camilla.Fabris@wolterskluwer.com"
message.CC = "marco.bitossi@wolterskluwer.com;Loredana.Montagna@wolterskluwer.com"
message.Subject = 'Aggiornamento ordini BPM'
message.HTMLBody = html_body
for attachment in attachments:
    message.Attachments.Add(Source=attachment)

message.Display()


